Pre-processing
* Visualization
1. Unix time
2. Encoding of categorical features
3. Temporal ordering
4. Aditional features:
- Previous event
- Next event
- Day of the week
- Time of day
- Event duration
5. Separate 80-20 
- Visualization
6. Get rid of overlap

In [1]:
import os
import psutil
import numpy as np  # import auxiliary library, typical idiom
import pandas as pd  # import the Pandas library, typical idiom
from pandas import read_csv
import statsmodels.api as sm
import time
import pm4py
from datetime import datetime
from datetime import date
from datetime import datetime
from datetime import timedelta

from numba import jit

from sklearn.linear_model import LinearRegression  # for linear regression
from sklearn import linear_model
from sklearn.cluster import KMeans  # for clustering
from sklearn.tree import DecisionTreeClassifier  # for decision tree mining
from sklearn.metrics import mean_absolute_error, confusion_matrix, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf 
import statsmodels.api as sm
from statsmodels.graphics.gofplots import ProbPlot
from matplotlib import pyplot
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

In [2]:
file_export = 'export2018.csv'
data = pd.read_csv(file_export)

In [3]:
data = data.sort_values(by=['case','startTime'])

In [4]:
#Duration
@jit(parallel = True)
def calculator_nb(case, startTime):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break

        if (case[idx + 1] == case[idx]):
            res[idx] = startTime[idx + 1]
        else:
            res[idx] = startTime[idx]

        idx+=1
    return res

data['completeTime'] = calculator_nb(data['case'].values, data['startTime'].values)
data.at[317373, 'completeTime'] = data.at[317373, 'startTime']

data['startTime'] =  pd.to_datetime(data['startTime'])
data['completeTime'] =  pd.to_datetime(data['completeTime'])
data['duration'] = data['completeTime'] - data['startTime']
#to turn duration into seconds:
duration = data['duration']
duration = duration / np.timedelta64(1, 's')
data['duration'] = duration

C:\Users\chbak\AppData\Local\Temp/ipykernel_1744/696655479.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_1744\696655479.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\chbak\AppData\Local\Temp/ipykernel_1744/696655479.py:2: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_1744\696655479.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "calculator_nb" was compiled in object mode without forceobj=Tr

In [5]:
#Next event
@jit(parallel = True)
def calculator_nb(case, event):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx + 1] == case[idx]):
            res[idx] = event[idx + 1]

        idx+=1
    return res

data['next_event'] = calculator_nb(data['case'].values, data['event'].values)

C:\Users\chbak\AppData\Local\Temp/ipykernel_1744/1360192189.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_1744\1360192189.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\chbak\AppData\Local\Temp/ipykernel_1744/1360192189.py:2: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_1744\1360192189.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "calculator_nb" was compiled in object mode without forceob

In [6]:
#Previous event
@jit(parallel = True)
def calculator_nb(case, event):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx + 1] == case[idx]):
            res[idx + 1] = event[idx]

        idx+=1
    return res

data['prev_event'] = calculator_nb(data['case'].values, data['event'].values)

C:\Users\chbak\AppData\Local\Temp/ipykernel_1744/1972138414.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_1744\1972138414.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\chbak\AppData\Local\Temp/ipykernel_1744/1972138414.py:2: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_1744\1972138414.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "calculator_nb" was compiled in object mode without forceob

In [7]:
#Removing null values
data['next_event'] = data['next_event'].fillna(value='None')
data['prev_event'] = data['prev_event'].fillna(value='None')

In [8]:
#unix time
pd.set_option('display.float_format', lambda x: '%.3f' % x)

data['startTime'] = pd.to_datetime(data['startTime'], dayfirst=True)
unixTransform = lambda x: time.mktime(x.timetuple())
data["UNIX_starttime"] = data["startTime"].apply(unixTransform).astype(int)

data['completeTime'] = pd.to_datetime(data['completeTime'], dayfirst=True)
unixTransform = lambda x: time.mktime(x.timetuple())
data["UNIX_completeTime"] = data["completeTime"].apply(unixTransform).astype(int)

#data['REG_DATE'] = pd.to_datetime(data['REG_DATE'], dayfirst=True)
#unixTransform = lambda x: time.mktime(x.timetuple())
#data["UNIX_REG_DATE"] = data["REG_DATE"].apply(unixTransform).astype(int)

#print(data)

In [9]:
#Day of the week
data['weekday'] = data['startTime'].dt.dayofweek

In [10]:
#encoding of categorical data
ordinal_encoder = OrdinalEncoder()
label_encoder = LabelEncoder()
data['enc_event'] = ordinal_encoder.fit_transform(data[['event']]).astype(int)

In [11]:
#ensure we have acces to orignal indexing to keep track of the order of events in a process
data['original index'] = data.index

#sorting on time
data.sort_values(by = "UNIX_starttime", ignore_index=True)

,case,event,startTime,completeTime,penalty_JLP1,penalty_JLP3,number_parcels,penalty_JLP2,penalty_JLP5,year,...,org:resource,success,duration,next_event,prev_event,UNIX_starttime,UNIX_completeTime,weekday,enc_event,original index
0,980452d16c80c2c3,mail valid,2014-05-04 00:00:00.000,2015-04-24 00:00:00.000,False,False,32,False,False,2015,...,0;n/a,True,30672000.000,mail income,mail valid,1399154400,1429826400,6,26,63730
1,980452d16c80c2c3,mail valid,2014-05-04 00:00:00.000,2014-05-04 00:00:00.000,False,False,32,False,False,2015,...,0;n/a,True,0.000,mail valid,mail valid,1399154400,1399154400,6,26,63729
2,cb3425ce193199d7,mail valid,2014-05-04 00:00:00.000,2014-05-04 00:00:00.000,False,False,5,False,False,2015,...,0;n/a,True,0.000,mail valid,None,1399154400,1399154400,6,26,594649
3,980452d16c80c2c3,mail valid,2014-05-04 00:00:00.000,2014-05-04 00:00:00.000,False,False,32,False,False,2015,...,0;n/a,True,0.000,mail valid,None,1399154400,1399154400,6,26,63727
4,cb3425ce193199d7,mail valid,2014-05-04 00:00:00.000,2014-05-04 00:00:00.000,False,False,5,False,False,2015,...,0;n/a,True,0.000,mail valid,mail valid,1399154400,1399154400,6,26,594651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2514261,8cd07a136f401349,calculate,2018-01-19 12:27:24.824,2018-01-19 12:27:24.824,False,False,37,False,False,2016,...,75992a,True,0.000,None,begin editing,1516361244,1516361244,4,8,1020230
2514262,dc56a531f5aeb3b0,save,2018-01-19 12:51:34.000,2018-01-19 12:51:34.000,False,False,49,False,False,2016,...,8d8538,True,0.000,None,finish payment,1516362694,1516362694,4,38,1553199
2514263,7b405bc5144c14c9,save,2018-01-19 12:57:05.000,2018-01-19 13:02:32.000,False,False,43,False,False,2016,...,8d8538,True,327.000,save,finish payment,1516363025,1516363352,4,38,1189623
2514264,7b405bc5144c14c9,save,2018-01-19 13:02:32.000,2018-01-19 13:02:32.000,False,False,43,False,False,2016,...,8d8538,True,0.000,None,save,1516363352,1516363352,4,38,1189624


In [12]:
#separation
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, shuffle=False)

In [13]:
#removing overlap - if case is in both datasets, remove

train_cases = train['case'].unique().tolist()
test_cases = test['case'].unique().tolist()

intersect_list = list(set(train_cases).intersection(test_cases))

In [14]:
#only removes first value in intersect list (needs modification for multiple overlaping values)

#train = train[train['case'] != intersect_list[0]]
#test = test[test['case'] != intersect_list[0]]

#works for more values
org_train = train.copy()
org_test = test.copy()
df_ordinal_encoder = LabelEncoder()
train=train.apply(df_ordinal_encoder.fit_transform)
test=test.apply(df_ordinal_encoder.fit_transform)

train = train[train['case'].isin(intersect_list) == False]
X_train_time = train.drop(columns='duration')
Y_train_time = train["duration"]
X_train_event = train.drop(columns=["next_event"])
Y_train_event = train["event"]

test = test[test['case'].isin(intersect_list) == False]
X_test_time = test.drop(columns='duration')
Y_test_time = test["duration"]
X_test_event = test.drop(columns=['next_event'])
Y_test_event = test["event"]

In [15]:
train.columns

Index(['case', 'event', 'startTime', 'completeTime', 'penalty_JLP1',
       'penalty_JLP3', 'number_parcels', 'penalty_JLP2', 'penalty_JLP5',
       'year', 'penalty_JLP7', 'penalty_JLP6', 'redistribution',
       'amount_applied1', 'amount_applied0', 'amount_applied3',
       'amount_applied2', 'identity:id', 'penalty_V5', 'payment_actual0',
       'payment_actual2', 'payment_actual1', 'penalty_B5F', 'payment_actual3',
       'penalty_B16', 'penalty_GP1', 'basic payment', 'penalty_AGP', 'area',
       'selected_manually', 'penalty_B3', 'penalty_B2', 'selected_risk',
       'penalty_B5', 'penalty_AVBP', 'penalty_B4', 'penalty_B6', 'penalty_ABP',
       'penalty_AVGP', 'penalty_C4', 'greening', 'rejected',
       'cross_compliance', 'penalty_C9', 'penalty_AVJLP', 'penalty_CC',
       'penalty_AVUVP', 'penalty_BGK', 'penalty_C16', 'penalty_BGP',
       'department', 'small farmer', 'risk_factor', 'applicant',
       'penalty_AUVP', 'penalty_amount2', 'penalty_BGKV', 'penalty_amount3',
  

Feature prediction for time and event based on KBest(z-scores)
note: don't run takes a significant time

In [67]:
from sklearn.feature_selection import SelectKBest

select = SelectKBest(k=10) # takes best 10 arguments 
z = select.fit_transform(X_train_time, Y_train_time)
filter = select.get_support()
print(np.extract(filter, train.columns))
# ['event' 'selected_random' 'note' 'eventid' 'activity' 'subprocess' 'org:resource' 'duration' 'next_event' 'weekday'] for time



select = SelectKBest(k=10) # takes best 10 arguments for event
z = select.fit_transform(X_train_event, Y_train_event)
filter = select.get_support()
print(np.extract(filter, train.columns)) 
# ['penalty_B2' 'penalty_AJLP' 'young farmer' 'note' 'eventid' 'docid' 'docid_uuid' 'success' 'duration' 'UNIX_completeTime'] for event

KeyboardInterrupt: 

In [32]:
train.columns

Index(['case', 'event', 'startTime', 'completeTime', 'penalty_JLP1',
       'penalty_JLP3', 'number_parcels', 'penalty_JLP2', 'penalty_JLP5',
       'year', 'penalty_JLP7', 'penalty_JLP6', 'redistribution',
       'amount_applied1', 'amount_applied0', 'amount_applied3',
       'amount_applied2', 'identity:id', 'penalty_V5', 'payment_actual0',
       'payment_actual2', 'payment_actual1', 'penalty_B5F', 'payment_actual3',
       'penalty_B16', 'penalty_GP1', 'basic payment', 'penalty_AGP', 'area',
       'selected_manually', 'penalty_B3', 'penalty_B2', 'selected_risk',
       'penalty_B5', 'penalty_AVBP', 'penalty_B4', 'penalty_B6', 'penalty_ABP',
       'penalty_AVGP', 'penalty_C4', 'greening', 'rejected',
       'cross_compliance', 'penalty_C9', 'penalty_AVJLP', 'penalty_CC',
       'penalty_AVUVP', 'penalty_BGK', 'penalty_C16', 'penalty_BGP',
       'department', 'small farmer', 'risk_factor', 'applicant',
       'penalty_AUVP', 'penalty_amount2', 'penalty_BGKV', 'penalty_amount3',
  

In [68]:
# Naive event NOT ACTUALLY DONE!!!!!!!!!!!!
data_baseline= test.copy()

@jit(parallel = True)
def calculator_pos(case):
    res = np.empty(len(case), dtype=object)
    idx = 0
    count=1
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx] == case[idx-1]):
            count+=1
            res[idx] = count
            
        else:
            count=1
            res[idx]=count

        idx+=1
    res[-1]=count+1
    return res

data_baseline["pos"] = calculator_pos(data_baseline['case'].values)

event_to_num = {}
list_of_events = train["event"].unique()
i=0
for event in list_of_events:
    event_to_num[str(event)] = i
    i += 1
event_to_num['None'] = i

pop=data_baseline.sort_values(by='pos')
pop['eventnum']=pop['enc_event']
pop2=pop.set_index('pos')
pop3=pop[['pos','eventnum']]
pop4=pop3.groupby(['pos', 'eventnum']).apply(pd.DataFrame.mode).reset_index(drop=True)
pop5=pop4.drop_duplicates(subset='pos')
ptenum= dict(zip(pop5.pos, pop5.eventnum))
num_to_event = {value:key for key, value in event_to_num.items()}
data_baseline['predicted_event_num'] = (data_baseline['pos']+1).map(ptenum)
data_baseline['predicted_event'] = (data_baseline['predicted_event_num']).map(num_to_event)
data_baseline_final=data_baseline.drop(['predicted_event_num'],axis=1)

next_task=[]
predicted_event=[]
for event in data_baseline_final['next_event']:
    next_task.append(str(event))
    

for case in data_baseline_final['predicted_event']:
    predicted_event.append(str(case))

accuracy_score(next_task,predicted_event)


test["naive_event"] = predicted_event

C:\Users\chbak\AppData\Local\Temp/ipykernel_5504/2508728517.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_pos" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_5504\2508728517.py", line 6:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\chbak\AppData\Local\Temp/ipykernel_5504/2508728517.py:4: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_pos" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_5504\2508728517.py", line 6:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "calculator_pos" was compiled in object mode without forc

In [70]:
# Naive time
#Sums up count for each event and the time each event takes
events_count = train.groupby("event")['duration'].agg('count')
event_duration_sum = train.groupby("event")['duration'].agg('sum')

#Computes average duration per event (basically our trained data that can be mapped onto test data)
duration_per_event = event_duration_sum / events_count 

test["naive_time"] = test['event'].map(duration_per_event)

In [77]:
X_train_event

,case,startTime,completeTime,penalty_JLP1,penalty_JLP3,number_parcels,penalty_JLP2,penalty_JLP5,year,penalty_JLP7,...,docid_uuid,org:resource,success,duration,prev_event,UNIX_starttime,UNIX_completeTime,weekday,enc_event,original index
136988,0,23,25,0,0,22,0,0,0,0,...,149739,8,1,659397,0,23,25,1,25,107180
136989,0,29,25,0,0,22,0,0,0,0,...,74304,8,1,0,26,29,25,0,26,107181
136990,0,29,25,0,0,22,0,0,0,0,...,149739,8,1,0,27,29,25,0,26,107182
136991,0,29,9470,0,0,22,0,0,0,0,...,74304,8,1,913113,27,29,9470,0,26,107183
136992,0,9474,9470,0,0,22,0,0,0,0,...,33412,96,1,0,27,9474,9470,4,23,107184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702996,35048,439389,452131,0,0,32,0,0,0,0,...,162559,95,1,396190,7,373184,375414,2,1,564053
702997,35048,452135,465177,0,0,32,0,0,0,0,...,162559,95,1,260636,2,375418,377313,3,6,564054
702998,35048,465181,472119,0,0,32,0,0,0,0,...,162559,95,1,230412,7,377317,378262,4,1,564055
702999,35048,472123,495619,0,0,32,0,0,0,0,...,162559,95,1,962345,2,378266,389200,4,6,564056


In [75]:
# Random forest event
DT = DecisionTreeClassifier()
RF = RandomForestClassifier()


# DT_fit = DT.fit(X_train_event, Y_train_event)
RF_fit = RF.fit(X_train_event['event'], Y_train_event)

# DT_pred = DT_fit.predict(X_test_event)
RF_pred = RF_fit.predict(X_test_event['event'])

KeyError: 'event'

In [17]:
#visualization (no need to include in main code, but might be useful)
log = pm4py.read_xes('bpi2018.xes')

process_tree = pm4py.discover_tree_inductive(log)
pm4py.view_process_tree(process_tree)

FileNotFoundError: [Errno 2] No such file or directory: 'bpi2018.xes'

In [ ]:
#separation visualisation

g = sns.scatterplot(x="UNIX_starttime", y="case", hue="enc_event", data=data, palette='colorblind', legend=False)

#add lines for separation - horizontal and vertical


In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [54]:
listVal = train
listVal.columns

Index(['case', 'event', 'startTime', 'completeTime', 'penalty_JLP1',
       'penalty_JLP3', 'number_parcels', 'penalty_JLP2', 'penalty_JLP5',
       'year', 'penalty_JLP7', 'penalty_JLP6', 'redistribution',
       'amount_applied1', 'amount_applied0', 'amount_applied3',
       'amount_applied2', 'identity:id', 'penalty_V5', 'payment_actual0',
       'payment_actual2', 'payment_actual1', 'penalty_B5F', 'payment_actual3',
       'penalty_B16', 'penalty_GP1', 'basic payment', 'penalty_AGP', 'area',
       'selected_manually', 'penalty_B3', 'penalty_B2', 'selected_risk',
       'penalty_B5', 'penalty_AVBP', 'penalty_B4', 'penalty_B6', 'penalty_ABP',
       'penalty_AVGP', 'penalty_C4', 'greening', 'rejected',
       'cross_compliance', 'penalty_C9', 'penalty_AVJLP', 'penalty_CC',
       'penalty_AVUVP', 'penalty_BGK', 'penalty_C16', 'penalty_BGP',
       'department', 'small farmer', 'risk_factor', 'applicant',
       'penalty_AUVP', 'penalty_amount2', 'penalty_BGKV', 'penalty_amount3',
  

In [76]:
columnNames = ['case', 'event', 'UNIX_starttime', 'UNIX_completeTime', 'weekday'] # chose better features

listValSelected = listVal[columnNames]
listValSelected_prediction = train[columnNames]
listValSelected_prediction = listValSelected_prediction.values
listValDuration_prediction = org_train['duration']
listValDuration_prediction = listValDuration_prediction.values
listValDuration = listVal['duration'].values

listValSelected = listValSelected.values

In [77]:
# choose a number of time steps
n_steps = len(listValSelected[0])
# split into samples


n_features = 1
X = listValSelected.reshape((listValSelected.shape[0], listValSelected.shape[1], n_features))
y = listValDuration

# define model
model = Sequential()
model.add(LSTM(50, input_shape=(n_steps, n_features), return_sequences=False, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(50, activation='relu'))
# Dropout for regularization
# model.add(Dropout(0.5))
# model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X, y, epochs=1, verbose=1, workers=-1)

# demonstrate prediction
x_input = listValSelected_prediction

x_input = x_input.reshape((x_input.shape[0], n_steps, n_features))

62857/62857 [==============================] - 382s 6ms/step - loss: 174656913408.0000


In [78]:
yhat = model.predict(x_input.astype('float32'), verbose=2)
lenVal_len = len(listValDuration_prediction) 


62857/62857 - 88s - 88s/epoch - 1ms/step


AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [79]:

print(mean_absolute_error(listValDuration_prediction, yhat.flatten()[:lenVal_len])) # 504639.3534027153 seconds
# test_df["duration_prediction"] = yhat.flatten()[:len(test_df)]

504639.3534027153
